# Знакомство с нейронными сетями (решение задач с Fashion MNIST)

Дополнительный материал:

* Первая модель: набор данных [Fashion MNIST](https://habr.com/ru/post/454034/)

Это руководство поможет тебе обучить нейросеть, которая классифицирует изображения одежды, например, кроссовки и рубашки. Это нормально, если не все будет понятно сразу: это быстрый, ознакомительный обзор полной программы TensorFlow, где новые детали объясняются по мере их появления.

In [ ]:
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
    pass

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow и tf.keras
import tensorflow as tf
from tensorflow import keras

# Вспомогательные библиотеки
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

## Загружаем датасет Fashion MNIST



Это руководство использует датасет Fashion MNIST который содержит 70,000 монохромных изображений в 10 категориях. На каждом изображении содержится по одному предмету одежды в низком разрешении (28 на 28 пикселей):

<table>
  <tr><td>
    <img src="https://tensorflow.org/images/fashion-mnist-sprite.png"
         alt="Fashion MNIST sprite"  width="600">
  </td></tr>
  <tr><td align="center">
    <a href="https://github.com/zalandoresearch/fashion-mnist"> Образцы Fashion-MNIST</a> (Zalando, лицензия MIT).<br/>&nbsp;
  </td></tr>
</table>

Fashion MNIST предназначен для замены классического датасета MNIST который часто используют как "Hello, World" программ машинного обучения для компьютерного зрения. Датасет [MNIST](http://yann.lecun.com/exdb/mnist/) содержит изображения рукописных цифр (0, 1, 2, и т.д.) в формате идентичном формату изображений одежды которыми мы будем пользоваться здесь.

Это руководство для разнообразия использует Fashion MNIST, и еще потому, что это проблема немного сложнее чем обычный MNIST. Оба датасета относительно малы, и используются для проверки корректности работы алгоритма. Это хорошие отправные точки для тестирования и отладки кода.

Мы используем 60,000 изображений для обучения нейросети и 10,000 изображений чтобы проверить, насколько правильно сеть обучилась их классифицировать. Вы можете получить доступ к Fashion MNIST прямо из TensorFlow. Импортируйте и загрузите данные Fashion MNIST прямо из TensorFlow:


In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

Загрузка датасета возвращает четыре массива NumPy:

* Массивы `train_images` и `train_labels` являются *тренировочным сетом* — данными, на которых модель будет обучаться.
* Модель тестируется на *проверочном сете*, а именно массивах `test_images` и `test_labels`.

Изображения являются 28х28 массивами NumPy, где значение пикселей варьируется от 0 до 255. *Метки (labels)* - это массив целых чисел от 0 до 9. Они соответствуют *классам* одежды изображенной на картинках:

<table>
  <tr>
    <th>Label</th>
    <th>Class</th>
  </tr>
  <tr>
    <td>0</td>
    <td>T-shirt/top</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Trouser</td>
  </tr>
    <tr>
    <td>2</td>
    <td>Pullover</td>
  </tr>
    <tr>
    <td>3</td>
    <td>Dress</td>
  </tr>
    <tr>
    <td>4</td>
    <td>Coat</td>
  </tr>
    <tr>
    <td>5</td>
    <td>Sandal</td>
  </tr>
    <tr>
    <td>6</td>
    <td>Shirt</td>
  </tr>
    <tr>
    <td>7</td>
    <td>Sneaker</td>
  </tr>
    <tr>
    <td>8</td>
    <td>Bag</td>
  </tr>
    <tr>
    <td>9</td>
    <td>Ankle boot</td>
  </tr>
</table>

Каждому изображению соответствует единственная метка. Так как *названия классов* не включены в датасет, сохраним их тут для дальнейшего использования при построении изображений:

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## Изучите данные



Давайте посмотрим на формат данных перед обучением модели. Воспользовавшись `shape` мы видим, что в тренировочном датасете 60,000 изображений, каждое размером 28 x 28 пикселей:

In [ ]:
train_images.shape

Соответственно, в тренировочном сете 60,000 меток:

In [ ]:
len(train_labels)

Каждая метка это целое число от 0 до 9:

In [ ]:
train_labels

Проверочный сет содержит 10,000 изображений, каждое - также 28 на 28 пикселей:

In [ ]:
test_images.shape

И в проверочном сете - ровно 10,000 меток:

In [ ]:
len(test_labels)

## Предобработайте данные


Данные должны быть предобработаны перед обучением нейросети. Если вы посмотрите на первое изображение в тренировочном сете вы увидите, что значения пикселей находятся в диапазоне от 0 до 255:

In [ ]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

Мы масштабируем эти значения к диапазону от 0 до 1 перед тем как скормить их нейросети. Для этого мы поделим значения на 255. Важно, чтобы тренировочный сет и проверочный сет были предобработаны одинаково:

In [ ]:
train_images = train_images / 255.0

test_images = test_images / 255.0

Чтобы убедиться, что данные в правильном формате и мы готовы построить и обучить нейросеть, выведем на экран первые 25 изображений из тренировочного сета и отобразим под ними наименования их классов.

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

## Постройте модель

Построение модели нейронной сети требует правильной конфигурации каждого слоя, и последующей компиляции модели.


### Настройте слои




В Keras, вы собираете **слои** (`layers`) для построения **моделей** (`models`). Модель это (обычно) граф слоев. Наиболее распространенным видом модели является **стек слоев**: модель `tf.keras.Sequential`.

Большая часть глубокого обучения состоит из соединения в последовательность простых слоев. Большинство слоев, таких как `tf.keras.layers.Dense`, имеют параметры, которые настраиваются во время обучения.

Типичная полносвязанная сеть (т.е. многослойный перцептрон):

```python
from tensorflow.keras import layers

model = tf.keras.Sequential()
# Добавим к модели полносвязный слой с 64 узлами:
model.add(layers.Dense(64, activation='relu'))
# Добавим другой слой:
model.add(layers.Dense(64, activation='relu'))
# Добавим слой softmax с 10 выходами:
model.add(layers.Dense(10, activation='softmax'))
```




In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

**Первый слой этой сети** - `tf.keras.layers.Flatten`, преобразует формат изображения из двумерного массива (`28 на 28 пикселей`) в одномерный (размерностью `28 * 28 = 784 пикселя`). Слой извлекает строки пикселей из изображения и выстраивает их в один ряд. Этот слой не имеет параметров для обучения; он только **переформатирует данные**.

После разложения пикселей, нейросеть содержит два слоя `tf.keras.layers.Dense`. Это **полносвязные нейронные слои**. Первый Dense слой состоит из `128 узлов (или нейронов)`. Второй (и последний) `10-узловой softmax` слой возвращает массив из **10 вероятностных оценок** дающих в сумме 1. Каждый узел содержит оценку указывающую вероятность принадлежности изображения к одному из 10 классов.

Доступно много разновидностей слоев `tf.keras.layers`. Большинство из них используют общий конструктор аргументов:

* `activation`: Установка функции активации для слоя. В этом параметре указывается имя встроенной функции или вызываемый объект. У параметра нет значения по умолчанию.
* `kernel_initializer` и `bias_initializer`: Схемы инициализации создающие веса слоя (ядро и сдвиг). В этом параметре может быть имя или вызываемый объект. По умолчанию используется инициализатор "Glorot uniform". [Инициализации](https://keras.io/initializers/) определяют способ задания начальных случайных весов слоев Keras.
* `kernel_regularizer` и `bias_regularizer`: Схемы регуляризации добавляемые к весам слоя (ядро и сдвиг), такие как L1 или L2 регуляризации. По умолчанию регуляризация не устанавливается.

### Суть [Softmax](https://alexanderkurakin.blogspot.com/2018/08/softmax.html)


Вспомним, что логистическая регрессия производит результаты между 0 и 1. Например, вывод логистической регрессии равный 0.8 от классификатора писем предполагает, что шанс того, что письмо спам равен `80%`, а шанс того, что письмо не спам равен `20%`. Очевидно, что сумма вероятностей того, что письмо будет спамом или не спамом будет равна `1.0`.

**Софтмакс (Softmax)** расширяет эту идею в мульти-классовый мир. То есть, софтмакс назначает дробные вероятности каждому классу в мультиклассовой проблеме. Эти дробные вероятности должны составлять вместе `1.0`. Это дополнительное ограничение помогает тренировке сойтись быстрее.


<table>
  <tr><td>
    <img src="https://1.bp.blogspot.com/-hVj8epcHJgo/W4mC2Fj7feI/AAAAAAAAAlM/tVpOqFA3M5E-nnYodzabauB4a2qQTbnKACPcBGAYYCw/s1600/softmax.png"
         alt="Fashion MNIST sprite"  width="394" height="237">
  </td></tr>
  <tr><td align="center">
    <a href="https://alexanderkurakin.blogspot.com/2018/08/softmax.html"> Сумма всех выходных сигналов равна 1.<br/>
  </td></tr>
</table>

Софтмакс реализован через слой нейронной сети прямо перед слоем вывода. Софтмакс слой должен иметь такое же количество узлов как слой вывода.

<table>
  <tr><td>
    <img src="https://3.bp.blogspot.com/-N3tsLE1P8VU/W4mC-sXbefI/AAAAAAAAAlI/GYXXXP_iytccCnbwOuN3Daq6PF9kowPDQCLcBGAs/s1600/softmax2.png"
         alt="Fashion MNIST sprite"  width="332" height="251">
  </td></tr>
  <tr><td align="center">
    <a href="https://alexanderkurakin.blogspot.com/2018/08/softmax.html"> Сумма всех выходных сигналов равна 1.<br/>
  </td></tr>
</table>

Если очень хочется понять `softmax`: [sim0nsays Лекция 2.1 - Softmax](https://youtu.be/64nwedrxjPo).

## Скомпилируйте модель




Прежде чем модель будет готова для обучения, нам нужно указать еще несколько параметров. Они добавляются на шаге compile модели:

* **Функция потерь** (`Loss function`) — измеряет точность модели во время обучения. Мы хотим минимизировать эту функцию чтоб "направить" модель в верном направлении.
* **Оптимизатор** (`Optimizer`) — показывает каким образом обновляется модель на основе входных данных и функции потерь.
* **Метрики** (`Metrics`) — используются для мониторинга тренировки и тестирования модели. Наш пример использует метрику accuracy равную доле правильно классифицированных изображений.

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Обучите модель



Обучение модели нейронной сети требует выполнения следующих шагов:

1. Подайте тренировочный данные в модель. В этом примере тренировочные данные это массивы `train_images` и `train_labels`.
1. Модель учится ассоциировать изображения с правильными классами.
1. Мы просим модель сделать прогнозы для проверочных данных, в этом примере массив `test_images`. Мы проверяем, соответствуют ли предсказанные классы меткам из массива `test_labels`.

Для начала обучения, вызовите метод `model.fit`, который называется так, поскольку "тренирует (fits)" модель на тренировочных данных:

In [ ]:
model.fit(train_images, train_labels, epochs=10)

В процессе обучения модели отображаются метрики потери (loss) и точности (accuracy).

## Оцените точность



Далее, сравните какую точность модель покажет на проверчном датасете:


In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nТочность на проверочных данных:', test_acc)

Полученная на проверочном сете точность оказалась немного ниже, чем на тренировочном. Этот разрыв между точностью на тренировке и тесте является **примером переобучения** (`overfitting`). Переобучение возникает, когда модель машинного обучения показывает на новых данных худший результат, чем на тех, на которых она обучалась.

## Сделайте предсказания



Теперь, когда модель обучена, мы можем использовать ее чтобы сделать предсказания по поводу нескольких изображений:

In [ ]:
predictions = model.predict(test_images)

Здесь полученная модель предсказала класс одежды для каждого изображения в проверочном датасете. Давайте посмотрим на первое предсказание:

In [ ]:
predictions[0]

Прогноз представляет из себя массив из 10 чисел. Они описывают "уверенность" (confidence) модели в том, насколько изображение соответствует каждому из 10 разных видов одежды. Мы можем посмотреть какой метке соответствует максимальное значение:

In [ ]:
np.argmax(predictions[0])

Мы можем построить график, чтобы взглянуть на полный набор из 10 предсказаний классов.

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

Давайте посмотрим на нулевое изображение, предсказание и массив предсказаний.


In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions,  test_labels)
plt.show()

In [ ]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions,  test_labels)
plt.show()

Давайте посмотрим несколько изображений с их прогнозами. Цвет верных предсказаний синий, а неверных - красный. Число это процент уверенности (от 100) для предсказанной метки. Отметим, что модель может ошибаться даже если она очень уверена.

In [ ]:
# Отображаем первые X тестовых изображений, их предсказанную и настоящую метки.
# Корректные предсказания окрашиваем в синий цвет, ошибочные в красный.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions, test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions, test_labels)
plt.show()

Наконец, используем обученную модель для предсказания класса на одном изображении.

In [ ]:
# Берем одну картинку из проверочного сета.
img = test_images[0]

print(img.shape)

Модели `tf.keras` оптимизированы для предсказаний на пакетах (batch) данных, или на множестве примеров сразу. Таким образом, даже если мы используем всего 1 картинку, нам все равно необходимо добавить ее в список:

In [ ]:
# Добавляем изображение в пакет данных, состоящий только из одного элемента.
img = (np.expand_dims(img,0))

print(img.shape)

Сейчас предскажем правильную метку для изображения:

In [ ]:
predictions_single = model.predict(img)

print(predictions_single)

In [ ]:
plot_value_array(0, predictions_single, test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)